In [0]:
import tensorflow as tf
import sys
import gc
from multiprocessing import Pool

In [0]:
!pip install pdf2image
!apt-get install poppler-utils 
import cv2
import numpy as np
from PIL import Image
import requests
import pandas as pd
from pdf2image import convert_from_path, convert_from_bytes

df = pd.read_csv('omeka_files.csv', delimiter=",")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.10).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
urls = []
length = []
for i in df["original_filename"]:
    # Remove integrated files
    if len(i) == 76:
        urls.append(i)
print(len(urls))

10193


In [0]:
def is_it_map(t):

    # Read in the image_data
    if t == 0:
        image_data = tf.compat.v1.gfile.FastGFile('/content/test.jpg', 'rb').read()
    else:
        image_data = tf.compat.v1.gfile.FastGFile('/content/testgs.jpg', 'rb').read()

    # Loads label file, strips off carriage return
    label_lines = [line.rstrip() for line 
                      in tf.compat.v1.gfile.GFile("/content/retrained_labels.txt")]

    # Unpersists graph from file
    with tf.compat.v1.gfile.FastGFile("/content/retrained_graph.pb", 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

    with tf.compat.v1.Session() as sess:

        # Feed the image_data as input to the graph and get first prediction
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        
        predictions = sess.run(softmax_tensor, \
                {'DecodeJpeg/contents:0': image_data})
        
        # Sort to show labels of first prediction in order of confidence
        top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
        
        for node_id in top_k:
            human_string = label_lines[node_id]
            score = predictions[0][node_id]
            # print the prediction result in a readable format
            # print('%s (score = %.5f)' % (human_string, score))
    sess.close()
    
    return (predictions[0][label_lines.index("maps")])


In [0]:
def process_image(t):
    with Pool(1) as p:
        return p.apply(is_it_map, (t,))

In [0]:
def create_test_file(url, loc_page):

    score_max = 0
    
    r = requests.get(url) 
    
    with open("std.pdf",'wb') as f: 

        f.write(r.content) 

    stds = convert_from_path("std.pdf")

    for i in stds:
    
        imcv = cv2.cvtColor(np.asarray(i), cv2.COLOR_RGB2BGR)
        imcvgs = cv2.cvtColor(np.asarray(i), cv2.COLOR_BGR2GRAY)
        scale_percent = 40 # percent of original size
        width = int(imcv.shape[1] * scale_percent / 200)
        height = int(imcv.shape[0] * scale_percent / 200)
        dim = (width, height)
        # resize image
        test = cv2.resize(imcv, dim, interpolation = cv2.INTER_AREA)
        testgs = cv2.resize(imcvgs, dim, interpolation = cv2.INTER_AREA)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        # create image with/without grayscale
        cv2.imwrite( "test.jpg", test)
        cv2.imwrite( "testgs.jpg", testgs)

        result = process_image(0)
        gc.collect()
        result_gs = process_image(1)
        gc.collect()

        result = max(result, result_gs)

        if result > score_max:
            if result > 0.85:
                loc_page.append([stds.index(i)+1])
            score_max = result

    return score_max

In [0]:
scores=[]
n=0
for i in urls:
    n+=1
    loc_page = []
    highest = create_test_file(i, loc_page)
    if highest > 0.85:
        for j in loc_page:
            j.append(i)
            scores.append(j)

In [0]:
for i in scores:
    print(i)

In [0]:
with open('results.txt', 'w') as filehandle:
    for listitem in scores:
        filehandle.write('%s at page %s\n' % (listitem[1], listitem[0]))